In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from plotnine import *
from pybaseball import statcast
from pybaseball import statcast_batter
from pybaseball import statcast_batter_exitvelo_barrels
from pybaseball import statcast_batter_expected_stats
from pybaseball import cache
cache.enable()

In [2]:
df = statcast(start_dt='2021-01-01', end_dt='2022-01-01')

This is a large query, it may take a moment to complete
Skipping offseason dates
Skipping offseason dates


100%|█████████████████████████████████████████| 246/246 [02:47<00:00,  1.47it/s]


In [3]:
pd.set_option('display.max_columns', None)
df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
109,FF,2021-11-02,93.7,1.39,6.72,"Smith, Will",493329,519293,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,5,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,X,6,ground_ball,0,2,2021,0.57,1.21,0.19,2.68,<NA>,<NA>,488726,2,9,Bot,98.12,136.43,<NA>,<NA>,518595,<NA>,<NA>,-4.348927,-136.263269,-7.335202,8.101619,29.384843,-15.743652,3.37,1.53,5,93.8,-28,93.4,2112,6.1,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.39,0.071,0.064,0.0,1,0,0,2,69,3,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,146,0.0,-0.138
121,FF,2021-11-02,92.9,1.38,6.72,"Smith, Will",493329,519293,NaN,foul,<NA>,<NA>,<NA>,<NA>,5,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,S,<NA>,NaN,0,1,2021,0.9,1.34,0.01,2.12,<NA>,<NA>,488726,2,9,Bot,<NA>,<NA>,<NA>,<NA>,518595,<NA>,<NA>,-5.455433,-134.989926,-8.881689,12.188852,30.69001,-14.009571,3.37,1.53,156,75.0,15,92.6,2206,6.3,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,69,2,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,145,0.0,-0.047
125,FF,2021-11-02,93.1,1.35,6.73,"Smith, Will",493329,519293,NaN,called_strike,<NA>,<NA>,<NA>,<NA>,6,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,S,<NA>,NaN,0,0,2021,0.81,1.52,0.78,2.13,<NA>,<NA>,488726,2,9,Bot,<NA>,<NA>,<NA>,<NA>,518595,<NA>,<NA>,-3.230974,-135.201801,-9.255781,10.67848,31.699974,-11.7058,3.4,1.53,<NA>,<NA>,<NA>,92.5,2216,6.2,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,69,1,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,143,0.0,-0.042
131,FF,2021-11-02,94.6,1.31,6.73,"Smith, Will",670541,519293,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,5,Yordan Alvarez flies out to left fielder Eddie...,W,L,L,HOU,ATL,X,7,fly_ball,3,2,2021,0.85,1.27,-0.23,2.66,<NA>,<NA>,488726,1,9,Bot,68.5,86.79,<NA>,<NA>,518595,<NA>,<NA>,-5.901934,-137.422092,-7.652311,12.118242,35.102245,-14.577857,3.58,1.68,312,92.7,39,93.8,2263,6.3,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.25,0.026,0.046,0.0,1,0,0,3,68,6,4-Seam Fastball,0,7,0,7,7,0,0,7,Infield shift,Standard,140,-0.001,-0.386
142,FF,2021-11-02,93.6,1.31,6.8,"Smith, Will",670541,519293,NaN,ball,<NA>,<NA>,<NA>,<NA>,13,Yordan Alvarez flies out to left fielder Eddie...,W,L,L,HOU,ATL,B,<NA>,NaN,2,2,2021,0.9,1.43,-1.15,1.51,<NA>,<NA>,488726,1,9,Bot,<NA>,<NA>,<NA>,<NA>,518595,<NA>,<NA>,-8.265487,-135.578023,-10.936295,12.932808,31.370147,-12.168466,3.46,1.68,<NA>,<NA>,<NA>,92.8,2239,6.2,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,68,5,4-Seam Fastball,0,7,0,7,7,0,0,7,Infield shift,Standard,152,0.0,0.12


In [4]:
# Remove spring training games and playoffs
df = df[df.game_type != 'S']
df = df[df.game_type != 'F']
df = df[df.game_type != 'L']
df = df[df.game_type != 'D']
df = df[df.game_type != 'W']
df = df[df.game_type != 'E']

In [5]:
df[(df['launch_speed'] >= 95) & (df['events'].str.contains('out|play|sac|choice', regex=True))].batter.value_counts().head(10)

608070    208
592518    175
621566    173
571448    170
521692    165
646240    161
502671    159
518934    159
666182    158
665489    155
Name: batter, dtype: Int64

In [6]:
# Joining datasets to find player names that match player_id's
barrels = statcast_batter_exitvelo_barrels(2021, 100)
barrels.sort_values('barrels', ascending=False)

,last_name,first_name,player_id,attempts,avg_hit_angle,anglesweetspotpercent,max_hit_speed,avg_hit_speed,fbld,gb,max_distance,avg_distance,avg_hr_distance,ev95plus,ev95percent,barrels,brl_percent,brl_pa
118,Ohtani,Shohei,660271,350,16.6,35.4,119.0,93.6,100.4,85.9,470,197,416.0,187,53.6,78,22.3,12.2
13,Guerrero Jr.,Vladimir,665489,496,9.4,33.7,117.4,95.1,98.5,93.4,465,172,409.0,274,55.2,75,15.1,10.7
28,Perez,Salvador,521692,454,15.9,34.1,114.4,93.0,97.6,88.8,460,189,407.0,255,56.2,74,16.3,11.1
72,Judge,Aaron,592450,397,11.6,38.5,119.0,95.8,98.5,93.3,455,185,402.0,230,58.4,70,17.6,11.1
138,Tatis Jr.,Fernando,665487,329,13.8,33.1,116.6,93.9,99.9,87.9,477,184,416.0,183,55.6,70,21.3,12.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,Hernandez,Yonny,660634,113,4.1,36.3,106.3,83.1,85.8,81.3,391,128,NaN,18,15.9,1,0.9,0.6
391,Locastro,Tim,641796,107,14.3,31.8,107.1,82.1,85.1,79.4,395,154,366.0,18,16.8,1,0.9,0.6
310,Sierra,Magneuris,642423,160,7.7,25.6,103.1,82.3,83.2,85.0,348,122,NaN,30,18.8,0,0.0,0.0
390,Nogowski,John,596056,108,15.6,38.0,108.1,88.1,90.9,85.2,387,183,387.0,31,28.7,0,0.0,0.0


In [7]:
merged = df.merge(barrels,left_on='batter', right_on='player_id')

In [8]:
merged.pitch_name.value_counts()

4-Seam Fastball    220200
Slider             122174
Sinker              97552
Changeup            72959
Curveball           51244
Cutter              43474
Knuckle Curve       13175
Split-Finger        10172
Fastball              616
Knuckleball            72
Eephus                 49
Name: pitch_name, dtype: int64

In [9]:
merged[(merged['launch_speed'] >= 95) & (merged['events'].str.contains('out|play|sac|choice', regex=True))].batter.value_counts()

608070    208
592518    175
621566    173
571448    170
521692    165
         ... 
621011     13
519299     13
660634     12
676391     12
641645     10
Name: batter, Length: 404, dtype: Int64

In [10]:
merged = merged.rename(columns={' first_name':'first_name'})

In [11]:
merged['batter_name']= merged['first_name'] + ' ' + merged['last_name']

In [12]:
merged['hho'] = merged[(merged['launch_speed'] >= 95) & (merged['events'].str.contains('out|play|sac|choice', regex=True))].batter_name.value_counts().head(10)

In [13]:
merged = merged[merged.description != 'foul']

In [14]:
merged[(merged['launch_speed']>=95)].batter_name.value_counts()

 Vladimir Guerrero Jr.    274
 Salvador Perez           255
 Manny Machado            247
 Bo Bichette              241
 Paul Goldschmidt         239
                         ... 
 Max Schrock               25
 Eric Sogard               21
 Travis Jankowski          21
 Tim Locastro              18
 Yonny Hernandez           18
Name: batter_name, Length: 404, dtype: int64

In [15]:
merged = merged.rename(columns={'attempts':'bbe'})

In [16]:
hho = merged[(merged['estimated_ba_using_speedangle'] >= .500) & (merged['events'].str.contains('out|play|sac|choice', regex=True))].batter_name.value_counts().head(10).to_frame().reset_index()

In [17]:
merged[(merged['estimated_ba_using_speedangle']> .500)].events.value_counts()

single                       13304
field_out                     7014
home_run                      4504
double                        4478
triple                         321
grounded_into_double_play      182
force_out                      168
double_play                    164
sac_fly                        151
field_error                     93
fielders_choice                 19
fielders_choice_out             11
sac_fly_double_play              3
triple_play                      1
Name: events, dtype: int64

In [18]:
merged[(merged['estimated_ba_using_speedangle']> .500) & (merged['events'].str.contains('out|play|sac|choice', regex=True))].events.value_counts()

field_out                    7014
grounded_into_double_play     182
force_out                     168
double_play                   164
sac_fly                       151
fielders_choice                19
fielders_choice_out            11
sac_fly_double_play             3
triple_play                     1
Name: events, dtype: int64

In [19]:
merged[(merged['estimated_ba_using_speedangle']> .495)].launch_speed.value_counts().head(10)

104.6    216
101.6    208
104.9    208
102.1    208
104.7    204
104.0    202
104.2    201
102.5    200
103.5    199
105.6    198
Name: launch_speed, dtype: Int64

In [20]:
hh = merged[merged['launch_speed']>=95]

In [21]:
hh.estimated_ba_using_speedangle.mean()

0.49536139012112995

In [22]:
merged.events.value_counts()

field_out                     63921
strikeout                     35388
single                        22875
walk                          13739
double                         7235
home_run                       5571
force_out                      3321
grounded_into_double_play      3003
hit_by_pitch                   1889
field_error                    1123
sac_fly                        1044
triple                          606
fielders_choice                 354
double_play                     350
sac_bunt                        253
fielders_choice_out             243
caught_stealing_2b              133
strikeout_double_play            96
other_out                        34
sac_fly_double_play              16
catcher_interf                   11
wild_pitch                        9
pickoff_1b                        6
caught_stealing_3b                6
pickoff_2b                        6
triple_play                       5
pickoff_caught_stealing_2b        4
caught_stealing_home        

In [23]:
merged[merged['events']=='hit_by_pitch'].batter_name.value_counts().head(20)

 Ty France            27
 Mark Canha           27
 Anthony Rizzo        23
 Jonathan India       23
 José Abreu           22
 Josh Harrison        18
 Kyle Farmer          18
 Will Smith           18
 Edmundo Sosa         17
 Victor Robles        16
 Nick Solak           15
 Bradley Zimmer       15
 Willson Contreras    14
 Randy Arozarena      14
 Salvador Perez       13
 C.J. Cron            13
 Tyler O'Neill        13
 Javier Báez          13
 Kolten Wong          13
 Starling Marte       13
Name: batter_name, dtype: int64

In [24]:
merged.events.value_counts()

field_out                     63921
strikeout                     35388
single                        22875
walk                          13739
double                         7235
home_run                       5571
force_out                      3321
grounded_into_double_play      3003
hit_by_pitch                   1889
field_error                    1123
sac_fly                        1044
triple                          606
fielders_choice                 354
double_play                     350
sac_bunt                        253
fielders_choice_out             243
caught_stealing_2b              133
strikeout_double_play            96
other_out                        34
sac_fly_double_play              16
catcher_interf                   11
wild_pitch                        9
pickoff_1b                        6
caught_stealing_3b                6
pickoff_2b                        6
triple_play                       5
pickoff_caught_stealing_2b        4
caught_stealing_home        

In [25]:
merged[merged['events'].str.contains('out|single|double|triple|run|fielders|error', regex=True, na=False)].batter_name.value_counts()

 Whit Merrifield       664
 Marcus Semien         652
 Tommy Edman           641
 Bo Bichette           640
 Isiah Kiner-Falefa    635
                      ... 
 Travis Jankowski      131
 Austin Wynns          130
 John Nogowski         129
 Max Schrock           125
 Ernie Clement         121
Name: batter_name, Length: 404, dtype: int64

In [26]:
hho

,index,batter_name
0,Freddie Freeman,52
1,David Fletcher,49
2,Manny Machado,47
3,Michael Brantley,45
4,José Ramírez,43
5,Pete Alonso,43
6,Andrew Benintendi,42
7,Jeimer Candelario,41
8,Tommy Edman,41
9,Dominic Smith,40


In [27]:
!pip install plotly

In [28]:
df

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,fielder_2,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
2888,FF,2021-10-03,92.3,1.4,6.8,"Smith, Will",596019,519293,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,1,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,X,5,ground_ball,1,2,2021,0.69,1.38,-0.69,2.83,<NA>,<NA>,<NA>,2,9,Top,77.54,155.31,<NA>,<NA>,518595,<NA>,<NA>,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,3.32,1.51,13,95.2,-13,92.8,2330,6.5,632254,519293,518595,663586,645277,501303,606115,592696,628338,594807,54.03,0.174,0.158,0.0,1,0,0,2,61,4,4-Seam Fastball,5,0,0,5,0,5,0,5,Infield shift,Strategic,148,0.0,-0.073
3021,SL,2021-10-03,80.6,1.6,6.64,"Smith, Will",596019,519293,NaN,foul,<NA>,<NA>,<NA>,<NA>,4,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,<NA>,NaN,1,1,2021,-0.77,0.48,-0.71,2.62,<NA>,<NA>,<NA>,2,9,Top,<NA>,<NA>,<NA>,<NA>,518595,<NA>,<NA>,-3.700232,-117.430885,-3.266842,-6.531123,19.79339,-27.369114,3.32,1.51,108,75.3,75,81.2,2254,6.4,632254,519293,518595,663586,645277,501303,606115,592696,628338,594807,54.15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,61,3,Slider,5,0,0,5,0,5,0,5,Infield shift,Strategic,315,0.0,-0.027
3220,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,NaN,foul,<NA>,<NA>,<NA>,<NA>,5,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,<NA>,NaN,1,0,2021,-0.65,-0.51,-0.04,2.46,<NA>,<NA>,<NA>,2,9,Top,<NA>,<NA>,<NA>,<NA>,518595,<NA>,<NA>,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,3.32,1.51,157,83.5,65,75.2,1940,6.2,632254,519293,518595,663586,645277,501303,606115,592696,628338,594807,54.34,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,61,2,Curveball,5,0,0,5,0,5,0,5,Infield shift,Standard,328,0.0,-0.02
3302,CU,2021-10-03,75.0,1.53,6.83,"Smith, Will",596019,519293,NaN,ball,<NA>,<NA>,<NA>,<NA>,12,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,B,<NA>,NaN,0,0,2021,-0.69,-0.69,2.1,3.89,<NA>,<NA>,<NA>,2,9,Top,<NA>,<NA>,<NA>,<NA>,518595,<NA>,<NA>,2.37583,-109.20583,2.277617,-5.902656,19.427562,-38.284747,3.31,1.48,<NA>,<NA>,<NA>,74.5,2017,5.9,632254,519293,518595,663586,645277,501303,606115,592696,628338,594807,54.61,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,61,1,Curveball,5,0,0,5,0,5,0,5,Standard,Standard,330,0.0,0.016
3577,FF,2021-10-03,91.2,1.49,6.66,"Smith, Will",607043,519293,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,4,"Brandon Nimmo grounds out, second baseman Ozzi...",R,L,L,ATL,NYM,X,4,ground_ball,1,0,2021,0.63,1.28,-0.31,2.8,<NA>,<NA>,<NA>,1,9,Top,144.83,145.72,<NA>,<NA>,518595,<NA>,<NA>,-5.868477,-132.500539,-6.486796,8.700586,30.11769,-15.941174,3.49,1.6,9,93.3,-18,90.9,2281,6.3,632254,519293,518595,663586,645277,501303,606115,592696,628338,594807,54.15,0.1,0.09,0.0,1,0,0,2,60,2,4-Seam Fastball,5,0,0,5,0,5,0,5,Standard,Standard,143,0.002,-0.189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [29]:
merged[(merged['pitch_type']=='FF')].release_speed.mean()

93.65293347860927

In [30]:
# df['launch_angle'] = np.floor(pd.to_numeric(df['launch_angle'], errors='coerce')).astype('Int64')

In [31]:
# df['launch_speed'] = np.floor(pd.to_numeric(df['launch_speed'], errors='coerce')).astype('Int64')

In [32]:
# df['estimated_ba_using_speedangle'] = np.floor(pd.to_numeric(df['estimated_ba_using_speedangle'], errors='coerce')).astype('Int64')

In [33]:
df = df.loc[:,['launch_speed', 'launch_angle', 'estimated_ba_using_speedangle']]

In [34]:
df = df.dropna()

In [35]:
df.dtypes

launch_speed                     Float64
launch_angle                       Int64
estimated_ba_using_speedangle    Float64
dtype: object

In [36]:
df

,launch_speed,launch_angle,estimated_ba_using_speedangle
2888,95.2,-13,0.174
3577,93.3,-18,0.1
2360,75.3,65,0.0
3704,96.8,18,0.409
2478,91.8,29,0.109
...,...,...,...
3802,80.3,-40,0.094
1754,71.1,-37,0.071
2777,38.8,28,0.229
3187,96.3,8,0.577


In [38]:
df.to_csv('xba.csv')

In [ ]:
df.sort_v